In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
import numpy as np

In [3]:
def add_centroid_points(input_geojson, output_geojson):
    """
    Добавляет точки в центроиды полигонов с разными маркерами для каждого класса
    
    Args:
        input_geojson (str): путь к входному GeoJSON файлу
        output_geojson (str): путь к выходному GeoJSON файлу
    """
    
    # Загружаем данные
    gdf = gpd.read_file(input_geojson)
    
    # Проверяем наличие необходимых колонок
    if 'class' not in gdf.columns and 'class_name' not in gdf.columns:
        raise ValueError("В данных должна быть колонка 'class' или 'class_name'")
    
    # Определяем колонку с классами
    class_column = 'class' if 'class' in gdf.columns else 'class_name'
    
    # Создаем словарь для маркеров разных классов
    class_markers = {
        'forest': 'o',      # круг для лесов
        'building': 's',    # квадрат для зданий
        'water': '^',       # треугольник для воды
        'road': 'D',        # ромб для дорог
        'field': 'v',       # треугольник вниз для полей
        # добавьте другие классы по необходимости
    }
    
    # Создаем список для хранения новых точечных объектов
    centroid_points = []
    
    # Проходим по всем полигонам
    for idx, row in gdf.iterrows():
        polygon = row.geometry
        
        # Проверяем, что геометрия является полигоном
        if polygon.geom_type in ['Polygon', 'MultiPolygon']:
            
            # Получаем центроид
            centroid = polygon.centroid
            
            # Получаем класс объекта
            obj_class = row[class_column]
            
            # Определяем маркер для класса (по умолчанию 'o')
            marker = class_markers.get(obj_class.lower() if isinstance(obj_class, str) else obj_class, 'o')
            
            # Создаем новую запись для точки
            point_feature = {
                'geometry': centroid,
                'class': obj_class,
                'marker': marker,
                'original_id': idx,
                'area': polygon.area
            }
            
            # Добавляем атрибуты из исходных данных
            for col in gdf.columns:
                if col != 'geometry':
                    point_feature[col] = row[col]
            
            centroid_points.append(point_feature)
    
    # Создаем GeoDataFrame для точек
    gdf_points = gpd.GeoDataFrame(centroid_points, crs=gdf.crs)
    
    # Сохраняем результат
    gdf_points.to_file(output_geojson, driver='GeoJSON')
    
    print(f"Создано {len(centroid_points)} точечных объектов")
    print(f"Результат сохранен в: {output_geojson}")
    
    return gdf_points

In [4]:
def visualize_results(polygons_gdf, points_gdf, class_column='class'):
    """
    Визуализирует полигоны и центроидные точки
    
    Args:
        polygons_gdf: GeoDataFrame с полигонами
        points_gdf: GeoDataFrame с точками
        class_column (str): название колонки с классами
    """
    
    # Создаем график
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Визуализируем полигоны
    unique_classes = polygons_gdf[class_column].unique()
    
    for cls in unique_classes:
        # Полигоны
        class_polygons = polygons_gdf[polygons_gdf[class_column] == cls]
        class_polygons.plot(ax=ax, alpha=0.3, label=f'{cls} (полигоны)')
        
        # Точки
        class_points = points_gdf[points_gdf[class_column] == cls]
        if not class_points.empty:
            # Получаем маркер для этого класса
            marker = class_points.iloc[0]['marker']
            class_points.plot(ax=ax, marker=marker, markersize=50, 
                            label=f'{cls} (точки)', alpha=0.8)
    
    ax.set_title('Полигоны и центроидные точки по классам')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()